# Kazakhstan oil and gas production daily numbers

First attempt to extract oil and gas production daily figures.

Data source: http://www.iacng.kz/en/


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [1]:
import requests
page = requests.get("http://www.iacng.kz/en/")
if page.status_code == 200:
    print("OK")

OK


In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
# print(soup.prettify())

In [3]:
divs = soup.find_all('div', class_='bl-areas')
daily_stats_div = divs[0]
print(daily_stats_div)
title = daily_stats_div.find('div', class_='bl-areas-title').get_text()
title

<div class="bl-areas">
<div class="bl-areas-title"> <b>Daily indicators of oil and gas treatment of the RK</b>
<br/>
           21.10.2019</div>
<div class="values-outer">
<ul class="list-values">
<li>
<div class="bl-areas-text" style="text-align: left;"><span class="short_text" id="result_box" lang="en"><span class="">Oil and gas condensate, thousand</span> <span class="">tons</span></span></div>
<div class="bl-areas-value">267.7</div>
</li>
<li>
<div class="bl-areas-text"><span class="" id="result_box" lang="en"><span class="">Gas production, million cubic meters</span><span class=""></span></span></div>
<div class="bl-areas-value">171.0</div>
</li>
<li>
<div class="bl-areas-text"><span class="" id="result_box" lang="en"><span class="">Processing of oil at the refinery of the RK thousand tons</span></span></div>
<div class="bl-areas-value">48.9 
      <br/>
</div>
</li>
</ul>
</div>
</div>


' Daily indicators of oil and gas treatment of the RK\n\n           21.10.2019'

In [40]:
import re
date = None
if 'Daily indicators of oil and gas treatment of the RK' in title:
    # extract date
    date = re.search("([0-9]{2}\.[0-9]{2}\.[0-9]{4})", title).group()

date

'04.07.2019'

In [9]:
results = dict()
for stat in daily_stats_div.find_all('li'):
    title = stat.find('div', class_='bl-areas-text').get_text()
    value = stat.find('div', class_='bl-areas-value').get_text()
    results[title] = value

results

{'Oil and gas condensate, thousand tons': '267.7',
 'Gas production, million cubic meters': '171.0',
 'Processing of oil at the refinery of the RK thousand tons': '48.9 \r\n      \n'}

In [7]:
results['Oil and gas condensate, thousand tons']

'267.7'

In [47]:
test = """<div class="bl-areas">
<div class="bl-areas-title"> <b>Daily indicators of oil and gas treatment of the RK</b>
<br/>
           04.07.2019</div>
<div class="values-outer">
<ul class="list-values">
<li>
<div class="bl-areas-text" style="text-align: left;"><span class="short_text" id="result_box" lang="en"><span class="">Oil and gas condensate, outdoors.</span> <span class="">ton</span></span></div>
<div class="bl-areas-value">256.8</div>
</li>
<li>
<div class="bl-areas-text"><span class="" id="result_box" lang="en"><span class="">Gas production, million cubic meters.</span> <span class="">m</span></span></div>
<div class="bl-areas-value">160.9</div>
</li>
<li>
<div class="bl-areas-text"><span class="" id="result_box" lang="en"><span class="">Processing of oil at the refinery of the RK thousand tons</span></span></div>
<div class="bl-areas-value">49.9</div>
</li>
</ul>
</div>
</div>"""

s = BeautifulSoup(test, 'html.parser')


<div class="bl-areas">
 <div class="bl-areas-title">
  <b>
   Daily indicators of oil and gas treatment of the RK
  </b>
  <br/>
  04.07.2019
 </div>
 <div class="values-outer">
  <ul class="list-values">
   <li>
    <div class="bl-areas-text" style="text-align: left;">
     <span class="short_text" id="result_box" lang="en">
      <span class="">
       Oil and gas condensate, outdoors.
      </span>
      <span class="">
       ton
      </span>
     </span>
    </div>
    <div class="bl-areas-value">
     256.8
    </div>
   </li>
   <li>
    <div class="bl-areas-text">
     <span class="" id="result_box" lang="en">
      <span class="">
       Gas production, million cubic meters.
      </span>
      <span class="">
       m
      </span>
     </span>
    </div>
    <div class="bl-areas-value">
     160.9
    </div>
   </li>
   <li>
    <div class="bl-areas-text">
     <span class="" id="result_box" lang="en">
      <span class="">
       Processing of oil at the refinery of the RK

In [48]:
title = s.find('div', class_='bl-areas-title').get_text()
title

' Daily indicators of oil and gas treatment of the RK\n\n           04.07.2019'

In [62]:
new_date = '-'.join(date.split('.')[::-1])
new_date

'2019-07-04'

In [2]:
from scraper.core.utils import logger
import logging
logger.level = logging.DEBUG

In [5]:
from scraper.jobs.kz_iacng.job import KazakhstanDailyJob

kz = KazakhstanDailyJob(full_load=True)
kz.run()

2019-10-22 10:03:58,959 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): proxy.iea.org:8080
2019-10-22 10:03:59,566 - urllib3.connectionpool - DEBUG - http://proxy.iea.org:8080 "GET http://www.iacng.kz/en/ HTTP/1.1" 200 None
2019-10-22 10:03:59,601 - scraper.jobs.kz_iacng.job - DEBUG - Full load: skipping existing current date file to avoid duplicates: kz_iacng_daily_2019-10-21.html
2019-10-22 10:03:59,602 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta:8000
2019-10-22 10:03:59,874 - urllib3.connectionpool - DEBUG - http://vipenta:8000 "GET /dimension/source HTTP/1.1" 200 659356
2019-10-22 10:03:59,894 - scraper.core.utils - INFO - download_and_get_checksum: 4.002571105957031 ms
2019-10-22 10:03:59,895 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta:8000
2019-10-22 10:03:59,910 - urllib3.connectionpool - DEBUG - http://vipenta:8000 "GET /dimension/source?code=kz_iacng_daily_2019-07-08 HTTP/1.1" 200 731
2019-1

In [50]:
[vars(source) for source in kz.sources]

[{'code': 'kz_iacng_daily_2019-07-08',
  'url': 'http://www.iacng.kz/en/',
  'path': 'kz_iacng_daily_2019-07-08.html',
  'long_name': 'KAZAKHSTAN KZ_IACNG Oil&Condensate, Gas amd Refinery Daily Production - 2019-07-08',
  'meta_data': {'content': <div class="bl-areas">
   <div class="bl-areas-title"> <b>Daily indicators of oil and gas treatment of the RK</b>
   <br/>
              08.07.2019</div>
   <div class="values-outer">
   <ul class="list-values">
   <li>
   <div class="bl-areas-text" style="text-align: left;"><span class="short_text" id="result_box" lang="en"><span class="">Oil and gas condensate, outdoors.</span> <span class="">ton</span></span></div>
   <div class="bl-areas-value">260.0</div>
   </li>
   <li>
   <div class="bl-areas-text"><span class="" id="result_box" lang="en"><span class="">Gas production, million cubic meters.</span> <span class="">m</span></span></div>
   <div class="bl-areas-value">165.5</div>
   </li>
   <li>
   <div class="bl-areas-text"><span class="

In [44]:
kz.download_source(kz.sources[0])

In [6]:
kz.dynamic_dim

defaultdict(list,
            {'source': [{'code': 'kz_iacng_daily_20190709',
               'url': 'http://www.iacng.kz/en/',
               'path': 'kz_iacng_daily_20190709.html',
               'long_name': 'KAZAKHSTAN KZ_IACNG Oil&Condensate, Gas amd Refinery Daily Production - 20190709'},
              {'code': 'kz_iacng_daily_20190708',
               'url': 'http://www.iacng.kz/en/',
               'path': 'kz_iacng_daily_20190708.html',
               'long_name': 'KAZAKHSTAN KZ_IACNG Oil&Condensate, Gas amd Refinery Daily Production - 20190709'}],
             'provider': [{'code': 'KZ_IACNG',
               'long_name': 'Information-Analytical Centre of Oil and Gas - JSC',
               'url': 'http://www.iacng.kz/en/'}]})

In [46]:
df = kz.get_data_from_source(kz.sources[0])
df

AttributeError: 'KazakhstanDailyJob' object has no attribute 'get_data_from_source'

In [33]:
TON_TO_BBL_RATIO = 7.65

def convert_value(x):
    """
    Converts a measure in tons to barrels using Kazakhstan oil density.
    :param x: amount in tons
    :return: amount in barrels
    """
   
    if x['unit'] == 'T':
        x['value'] = x['value'] * TON_TO_BBL_RATIO / 1000
        x['unit'] = 'KBBL'
    elif x['unit'] == 'KT':
        x['value'] = x['value'] * TON_TO_BBL_RATIO
        x['unit'] = 'KBBL'
    else:
        x['value']
    
    return x

df.apply(convert_value, axis=1)

,flow,product,unit,period,value
0,SUPPLY,CRUDEOIL,KBBL,2019-07-08,1.989
0,SUPPLY,NATGAS,MCM,2019-07-08,165.500
0,REFINOBS,CRUDEOIL,KBBL,2019-07-08,278.460


In [22]:
import inspect
inspect.getmembers(kz, predicate=inspect.ismethod)

[('__init__',
  <bound method KazakhstanDailyJob.__init__ of <scraper.jobs.kz_iacng.job.KazakhstanDailyJob object at 0x00000047E0890208>>),
 ('download_and_get_checksum',
  <bound method Job.download_and_get_checksum of <scraper.jobs.kz_iacng.job.KazakhstanDailyJob object at 0x00000047E0890208>>),
 ('get_source_from_code',
  <bound method Job.get_source_from_code of <scraper.jobs.kz_iacng.job.KazakhstanDailyJob object at 0x00000047E0890208>>),
 ('get_sources',
  <bound method KazakhstanDailyJob.get_sources of <scraper.jobs.kz_iacng.job.KazakhstanDailyJob object at 0x00000047E0890208>>),
 ('insert_new_dynamic_dim',
  <bound method Job.insert_new_dynamic_dim of <scraper.jobs.kz_iacng.job.KazakhstanDailyJob object at 0x00000047E0890208>>),
 ('remove_existing_dynamic_dim',
  <bound method Job.remove_existing_dynamic_dim of <scraper.jobs.kz_iacng.job.KazakhstanDailyJob object at 0x00000047E0890208>>),
 ('rm_sources_up_to_date',
  <bound method Job.rm_sources_up_to_date of <scraper.jobs.kz_i

In [47]:
kz.transform()

In [48]:
kz.data

[{'flow': 'SUPPLY',
  'product': 'CRUDEOIL',
  'unit': 'KBBL',
  'period': '2019-07-08',
  'value': 1.989,
  'provider': 'KZ_IACNG',
  'area': 'KAZAKHSTAN',
  'frequency': 'Daily',
  'original': True},
 {'flow': 'SUPPLY',
  'product': 'NATGAS',
  'unit': 'MCM',
  'period': '2019-07-08',
  'value': 165.5,
  'provider': 'KZ_IACNG',
  'area': 'KAZAKHSTAN',
  'frequency': 'Daily',
  'original': True},
 {'flow': 'REFINOBS',
  'product': 'CRUDEOIL',
  'unit': 'KBBL',
  'period': '2019-07-08',
  'value': 278.46,
  'provider': 'KZ_IACNG',
  'area': 'KAZAKHSTAN',
  'frequency': 'Daily',
  'original': True},
 {'flow': 'SUPPLY',
  'product': 'CRUDEOIL',
  'unit': 'KBBL',
  'period': '2019-07-04',
  'value': 1.9645200000000003,
  'provider': 'KZ_IACNG',
  'area': 'KAZAKHSTAN',
  'frequency': 'Daily',
  'original': True},
 {'flow': 'SUPPLY',
  'product': 'NATGAS',
  'unit': 'MCM',
  'period': '2019-07-04',
  'value': 160.9,
  'provider': 'KZ_IACNG',
  'area': 'KAZAKHSTAN',
  'frequency': 'Daily',
 

In [57]:
import datetime
print(datetime.datetime.today().strftime('%Y-%m-%d'))


2019-07-09


In [59]:
path = 'kz_iacng_daily_20190709.html'
path.split('.')[0].split('_')[-1]

'20190709'

In [4]:
len('KAZAKHSTAN KZ_IACNG Oil&Condensate, Gas amd Refinery Daily Production - 20190709')

80

In [19]:
import pandas as pd
dtf = pd.DataFrame(kz.data)
dtf.sort_values(by='period', ascending=False)
dtf.drop_duplicates(subset=['flow', 'product', 'unit', 'period', 'value'], keep='first', inplace=True)

In [20]:
dtf

,area,flow,frequency,original,period,product,provider,source,unit,value
0,KAZAKHSTAN,SUPPLY,Daily,True,2019-07-04,CRUDEOIL,KZ_IACNG,kz_iacng_daily_20190707,KBBL,1.96452
1,KAZAKHSTAN,SUPPLY,Daily,True,2019-07-04,NATGAS,KZ_IACNG,kz_iacng_daily_20190707,MCM,160.90000
2,KAZAKHSTAN,REFINOBS,Daily,True,2019-07-04,CRUDEOIL,KZ_IACNG,kz_iacng_daily_20190707,KBBL,381.73500


In [2]:
MAIL_RECIPIENT = ['luisfernando.rosa@iea.org']
NEW_DATA_DAILY_RECIPIENT = ['toril.bosoni@iea.org', 'Anne.KLOSS@iea.org', 'peg.mackey@iea.org'] + MAIL_RECIPIENT
print(NEW_DATA_DAILY_RECIPIENT)

['toril.bosoni@iea.org', 'Anne.KLOSS@iea.org', 'peg.mackey@iea.org', 'luisfernando.rosa@iea.org']
